# 5강) BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

!pip install datasets
!pip install transformers

In [1]:
import os
import json

## 데이터셋 로딩


In [2]:
data_path  = "../../data/"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(dict.fromkeys([v["text"] for v in wiki.values()]))
print('context len :', len(corpus))

context len : 56737


In [3]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [6]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [7]:
from datasets import load_from_disk

dataset_dir = '../../data/train_dataset'
dataset = load_from_disk(dataset_dir)

training_dataset = dataset['train']#[sample_idx]
print(len(dataset['train']), len(training_dataset))

Negative sampling을 위한 negative sample들을 샘플링

In [10]:
# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break

print('[Positive context]')
print(p_with_neg[0], '\n')
print('[Negative context]')
print(p_with_neg[1], '\n', p_with_neg[2])

In [12]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')

In [13]:
max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

print(p_seqs['input_ids'].size())  #(num_example, pos + neg, max_len)

torch.Size([3952, 4, 512])


In [14]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [16]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

In [17]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

Train function 정의 후, 두개의 encoder fine-tuning 하기


In [18]:
def train(args, num_neg, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)  #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      #print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()
  return p_model, q_model

In [19]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01
)

In [20]:
p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

Epoch: 100%|██████████| 2/2 [29:40<00:00, 890.09s/it]


## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [77]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))
valid_corpus_idx = [np.where(corpus==e)[0].tolist()[0] for e in valid_corpus] # valid문서들의 전체문서 idx
valid_corpus_idx = np.array(valid_corpus_idx)

# Rank를 비교해서 검증

### Dense

In [137]:
answer_rank_list = []
for idx in tqdm(range(len(dataset['validation']))):
    # 1. query와 정답을 뽑아내기
    query = dataset['validation'][idx]['question']
    ground_truth = dataset['validation'][idx]['context']

    if not ground_truth in valid_corpus:
        valid_corpus.append(ground_truth) 
    # print(query)
    # print(ground_truth, '\n\n')

    # 2. passage encoder, question encoder을 이용해 dense embedding 생성
    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

        p_embs = []
        for p in valid_corpus:
            p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
            p_emb = p_encoder(**p).to('cpu').numpy()
            p_embs.append(p_emb)

    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    #print(p_embs.size(), q_emb.size())

    # 3. 생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    #print(dot_prod_scores.size())
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    #print(dot_prod_scores)
    #print(rank)

    # 4. rank에서 groundtruth와 비교
    rank_doc_idx = valid_corpus_idx[rank.tolist()] # rank된 문서들의 전체문서 idx
    ground_truth_doc_idx = np.where(corpus==ground_truth)[0].tolist()[0] # ground truth의 전체문서 idx
    answer_rank = np.where(rank_doc_idx==ground_truth_doc_idx)[0].tolist()[0] # 정답문서의 rank
    print(answer_rank)
    answer_rank_list.append(answer_rank)

  0%|          | 1/240 [00:04<18:18,  4.60s/it]

214


  1%|          | 2/240 [00:09<18:26,  4.65s/it]

56


  1%|▏         | 3/240 [00:13<18:17,  4.63s/it]

...

9


 75%|███████▌  | 181/240 [18:50<09:27,  9.62s/it]

64


 75%|███████▌  | 181/240 [18:52<06:09,  6.26s/it]


KeyboardInterrupt: 

In [ ]:
# 한 배치안에서 Negative 섞고, BM25, 
rank_doc_idx

In [135]:
answer_rank_list

[]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer(tokenizer=tokenizer.tokenize, ngram_range=(1, 2))#, max_features=50000)
p_embedding = tfidfv.fit_transform(corpus)

def get_topk_similarity(qeury_vec, k):
    result = qeury_vec * p_embedding.T
    result = result.toarray()

    doc_scores3 = np.partition(result, -k)[:, -k:][:, ::-1]
    ind = np.argsort(doc_scores3, axis=-1)[:, ::-1]
    doc_scores3 = np.sort(doc_scores3, axis=-1)[:, ::-1]
    doc_indices3 = np.argpartition(result, -k)[:, -k:][:, ::-1]
    r, c = ind.shape
    ind = ind + np.tile(np.arange(r).reshape(-1, 1), (1, c)) * c
    doc_indices3 = doc_indices3.ravel()[ind].reshape(r, c)

    return doc_scores3, doc_indices3

In [134]:
rank

tensor([ 23, 125, 124,  47,  76,  55,  40, 195, 160, 208,  54, 110,  93, 233,
        174, 232,  44, 158, 228,  12,  84,  41,  26, 227, 169, 101,  92, 216,
         62,  33,  85,  96, 191, 103, 177,  88,   7, 139,  14,   5,  17, 221,
        100,  13,  43, 218, 154, 157,  29,  60, 179, 176, 171,  64, 187, 196,
        204, 202,  19, 126,  50, 234, 105, 140, 225,  71, 188, 133,  87, 152,
        224,  10,  11,  99,  49,  39, 181,  72,  90, 127,  31, 143, 207, 163,
         78,  28, 166,  35, 197, 150, 213, 151,  16,  30, 111,  98,  91, 148,
         82, 215, 189, 167, 209, 205, 117,  38, 200,   1,  32, 220,  83, 159,
        165,   2, 114, 129, 156, 206, 130,  95, 104,  25,  74,  18, 122, 162,
        226, 223, 173, 184,  53, 146, 109, 211,   6, 145,  86, 155,  75,  37,
         56, 170, 190,  24, 168,  80, 194,   0,  73,  89, 118,  77, 132,  65,
         66,  61, 121,  68,  48, 137,  63, 199,  46,  81, 185, 134, 210,  70,
        183, 128, 164, 147, 172, 102, 186, 144,  21, 198, 178,  

In [121]:
valid_corpus_idx[rank.tolist()]

array([55500,  5153, 23027, 52576, 11676, 40413, 11939, 55828, 29185,
       24486, 48820,  4792, 23816, 23335, 26381, 54179,  4938,  4620,
       39395, 51965, 55243, 30932, 20503,  9353, 49488,  5263, 23667,
       16171,  4959, 10976, 32363, 22337, 34686,  5238, 31555, 23695,
        5762,  5081, 23517, 44248, 29373, 38315, 16121, 25833, 25992,
        7993, 55851, 35401, 18142,  4651, 52635, 48939, 29001, 33104,
       12290, 14631,  5287,  9613, 23291,  5006,  6183, 54652, 21801,
        5188, 12748, 42759, 51565, 52707, 29979,  5256, 21174,  5159,
       37389, 30877, 45254,  4846, 54997, 34100, 37030, 31148, 49540,
        4871, 42016,  8387, 26055,  4834, 49702, 26999, 28253,  4963,
       40091, 55663,  8658, 54074, 56430,  5269, 32220, 33013,  6925,
       14415, 15653,  4832, 35175, 34822, 55399, 32778, 13676, 37140,
        5177,  5259, 37476, 25018, 46731, 51048, 32530,  9735, 13194,
       23856,  4583, 12004, 37785,  9855, 28583, 14171, 36572, 40862,
        5103, 12069,

In [120]:
print(ground_truth_doc_idx)
print(valid_corpus_idx[rank.tolist()])

11891
[ 9353 40862 36572 35401 54997 14631 29373 44463 53651 53324 12290 37476
 54074  5068 49022  5139 25992  5048  5067 23816 26055 38315 23667 21751
 18935  4832  8658  4693 21801  5238  4834 32220 55699 34822 49502 28253
 55828 30722 26381 40413  4620 42566 15653 23335 25833 49449  5617 27704
 10976  6183 37574 55273 11017 12748  4919 14801  4930  4789 51965  5103
 52635 15015 32778  8538 44764  5159 47950 42569 26999 12031 55702 48820
  4792 14415  4651 44248  5705 37389 40091 12069 22337  7714 42467 20729
 37030  4959  4842 23695 49874 55756 42568 52190  4938 32363 25018  6925
 55663 29834 42016 24286 23598 31254 16383  5264 23856 23517  6315  5153
 34686 27392  8387 21658 20988 23027 32530 11891  4948 21562 23623  5269
 55399  5263 45254 39395 28583 31426 16711 31963 30775 41755 33104  5983
  5259  4804 11939 35358 49702 53318  4846  5081  5287  8142  5298 49488
 22206 49540 51001 55500 30877  4963  4583 34100 42209 42759 51565 54652
  9855 29979 18142 21571  5188  4953 55851  4

In [118]:
rank

tensor([ 23, 125, 124,  47,  76,  55,  40, 195, 160, 208,  54, 110,  93, 233,
        174, 232,  44, 158, 228,  12,  84,  41,  26, 227, 169, 101,  92, 216,
         62,  33,  85,  96, 191, 103, 177,  88,   7, 139,  14,   5,  17, 221,
        100,  13,  43, 218, 154, 157,  29,  60, 179, 176, 171,  64, 187, 196,
        204, 202,  19, 126,  50, 234, 105, 140, 225,  71, 188, 133,  87, 152,
        224,  10,  11,  99,  49,  39, 181,  72,  90, 127,  31, 143, 207, 163,
         78,  28, 166,  35, 197, 150, 213, 151,  16,  30, 111,  98,  91, 148,
         82, 215, 189, 167, 209, 205, 117,  38, 200,   1,  32, 220,  83, 159,
        165,   2, 114, 129, 156, 206, 130,  95, 104,  25,  74,  18, 122, 162,
        226, 223, 173, 184,  53, 146, 109, 211,   6, 145,  86, 155,  75,  37,
         56, 170, 190,  24, 168,  80, 194,   0,  73,  89, 118,  77, 132,  65,
         66,  61, 121,  68,  48, 137,  63, 199,  46,  81, 185, 134, 210,  70,
        183, 128, 164, 147, 172, 102, 186, 144,  21, 198, 178,  

In [116]:
answer_rank_list[-10:]

[112, 60, 159, 37, 94, 33, 57, 57, 0, 0]

In [96]:
k = 1
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 마르크스주의자들의 사상은? 

[Ground truth passage]
사회주의 혁명은 오로지 선진노동자계급에 기초한 계급투쟁으로서 이루어질 수 있다고 주장한 레닌의 노선은 본질적으로는 프랑스의 사회주의자인 오귀스트 블랑키(Auguste Blanqui)의 비밀결사주의와 동일하지 않다. 다음은 블랑키주의에 대한 레닌의 비판이다.\n블랑키주의는 계급투쟁을 긍정하는 이론이다. 그러나 블랑키주의는 프롤레타리아의 계급투쟁에 의거하지 않고 소수 인텔리겐차의 음모로써 인류가 임금노예제로부터 해방될 것을 기대한 것이다. 블랑키의 행동 지침은 부르주아 민주주의의 계급 모순을 지각한 혁명적 부르주아의 일반적인 경향이며, 노동계급에 의한 계급의식의 표출과는 무관한 것이다.|블라디미르 레닌, 『대회의 총결과에 붙여서』(1906년) \n동시에 레닌은 블랑키주의가 소수 지식인의 음모에 의한 혁명 방식이며, 소수에 의한 쿠데타와 다를 바가 없다고 비판하기도 하였다.\n\n그러나, 경제적 후진성과 러시아 정교회를 비롯한 여러 반동적 사상 조류가 극심했던 당시 러시아 사회의 특성을 고려하여, 소수 직업혁명가의 역량 확보를 강조하였고, 이 지점에서 블랑키의 사상과 밀접한 연관을 이루게 됐다. 특히 폭력혁명에 대한 긍정 및 합법 활동과 비합법 활동을 혁명의 성취라는 목적에 따라 적절히 배합해야 한다는 레닌의 주장은 블랑키의 주장과 상당히 흡사한 지점이다.\n\n특히, 당원의 지적 수련, 금욕적 생활, 사생취의(捨生取義) 정신을 강조했다는 점과, 일반적인 노동자계급과, 노동자계급을 지도하는 직업 혁명가의 뚜렷한 구분은 기존 마르크스주의자들과 달랐던 지점이다. 이러한 지점은 여러 학자들에 의해 블랑키주의의 영향을 받았다고 평가받는다. 공산주의 혁명가인 로자 룩셈부르크는 레닌과 블랑키의 차이는 지엽적이며, 본질적으로는 같은 것이라고 평가하였다. 이러한 비판은 러시아 10월 혁명이 “순수한 프롤레타리아 계급에 의한 혁명인가?”, 아니면 “소수 혁명적 지식인에 의한 쿠데타인

## sparse

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer(tokenizer=tokenizer.tokenize, ngram_range=(1, 2))#, max_features=50000)
p_embedding = tfidfv.fit_transform(corpus)

def get_topk_similarity(qeury_vec, k):
    result = qeury_vec * p_embedding.T
    result = result.toarray()

    doc_scores3 = np.partition(result, -k)[:, -k:][:, ::-1]
    ind = np.argsort(doc_scores3, axis=-1)[:, ::-1]
    doc_scores3 = np.sort(doc_scores3, axis=-1)[:, ::-1]
    doc_indices3 = np.argpartition(result, -k)[:, -k:][:, ::-1]
    r, c = ind.shape
    ind = ind + np.tile(np.arange(r).reshape(-1, 1), (1, c)) * c
    doc_indices3 = doc_indices3.ravel()[ind].reshape(r, c)

    return doc_scores3, doc_indices3

In [140]:
sparse_answer_rank_list = []
for idx in tqdm(range(len(dataset['validation']))):
    query = dataset['validation'][idx]['question']
    ground_truth = dataset['validation'][idx]['context']

    query_vec = tfidfv.transform([query])
    k=len(dataset['validation'])

    doc_scores, doc_indices = get_topk_similarity(query_vec, k)
    #print(doc_indices, doc_scores)
    # 4. rank에서 groundtruth와 비교
    ground_truth_doc_idx = np.where(corpus==ground_truth)[0].tolist()[0] # ground truth의 전체문서 idx
    #print(ground_truth_doc_idx)
    answer_rank = np.where(doc_indices[0]==ground_truth_doc_idx)[0].tolist()[0] # 정답문서의 rank
    #print(doc_indices)
    sparse_answer_rank_list.append(answer_rank)
    #break

 26%|██▋       | 63/240 [01:40<04:43,  1.60s/it]


IndexError: list index out of range

In [139]:
sparse_answer_rank_list

[1]

In [92]:
dense_rank = np.array(answer_rank_list).mean()
sparse_rank = np.array().mean()

112.05416666666666

#### Top-5개의 passage를 retrieve 하고 ground truth와 비교하기